In [1]:
import open3d as o3d
import numpy as np
import sys
import open3d.visualization.gui as gui


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
sys.path.insert(0,'sample')

In [3]:
from read_datas import ReadDatas

In [4]:
datas =  ReadDatas('Point Cloud Data\Corner.ply')

In [5]:
datas.gpf_ground_extraction()

In [6]:
ground = datas.np_to_o3d(datas.ground)


In [7]:
vis = o3d.visualization.VisualizerWithEditing()
vis.create_window(window_name='Open3D', visible=True)
vis.add_geometry(ground)
vis.run()
point = vis.get_picked_points()
geometry = vis.get_cropped_geometry()
vis.destroy_window()

[Open3D INFO] Picked point #15100 (-14., -13., -1.6) to add in queue.
[Open3D INFO] Picked point #13992 (-6.6, -13., -1.6) to add in queue.
[Open3D INFO] Picked point #19673 (-4.2, -13., -1.6) to add in queue.
[Open3D INFO] Picked point #35148 (-1.1, -12., -1.6) to add in queue.
[Open3D INFO] Picked point #60451 (4.1, -7.7, -1.5) to add in queue.
[Open3D INFO] Picked point #69243 (5.1, -2.3, -1.5) to add in queue.
[Open3D INFO] No point has been picked.
[Open3D INFO] No point has been picked.
[Open3D INFO] Picked point #70431 (5.0, 5.9, -1.5) to add in queue.


In [8]:
point

[15100, 13992, 19673, 35148, 60451, 69243, 70431]

In [9]:
def no_paves(ground):    
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window(window_name='Open3D', visible=True)
    vis.add_geometry(ground)
    vis.run()
    geometry = vis.get_cropped_geometry()
    vis.destroy_window()
    geometry = np.asarray(geometry.points)

    return geometry

In [10]:
geometrys = np.array([[0,0,0]])
yuncing = True


In [11]:

while yuncing:
    geometry = no_paves(ground)
    geometrys = np.concatenate((geometrys,geometry),axis=0)
    input_1 = input('是否继续')
    if input_1 == 'y' or input_1 == 'Y':
        yuncing = True
    else:
        yuncing = False

In [12]:
ground = ground.select_by_index(point, invert=True)

In [13]:
nndist = ground.compute_nearest_neighbor_distance()
nndist = np.array(nndist)
density = np.mean(nndist)

In [14]:
driving_point = datas.ground[point,:]

In [15]:
driving_distance = np.array([np.linalg.norm(driving_point[p] - driving_point[p+1]) for p in range(len(driving_point) - 1 )])


In [16]:
driving_track = np.array([[0,0,0]])

In [17]:
for i,d in enumerate(driving_distance):
    num = int(d / density)

    driving_track_x = np.linspace(driving_point[i,0],driving_point[i+1,0],num)
    driving_track_y = np.linspace(driving_point[i,1],driving_point[i+1,1],num)
    driving_track_z = np.linspace(driving_point[i,2],driving_point[i+1,2],num)
    new_points = np.array([driving_track_x,driving_track_y,driving_track_z]).T

    driving_track = np.concatenate((driving_track,new_points),axis=0)
    







In [18]:
driving_track = np.delete(driving_track,0,axis=0)


In [19]:
driving_track_o3d = datas.np_to_o3d(driving_track)


In [20]:
driving_track_o3d.paint_uniform_color([0,0,55])

[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]


PointCloud with 645 points.

In [21]:
o3d.visualization.draw_geometries([ground,driving_track_o3d])


In [22]:
ground += driving_track_o3d

In [23]:
neighbour_number = 1
kdtree = o3d.geometry.KDTreeFlann(driving_track_o3d)
number = len(ground.points)

point_neighbours = np.zeros((number, neighbour_number))
for ik in range(number):
    [_, idx, _] = kdtree.search_knn_vector_3d(ground.points[ik], neighbour_number)  # K近邻搜索
    point_neighbours[ik, :] = idx


In [24]:
ground_np = np.asarray(ground.points)
ground_nom = ground.estimate_normals()
ground_nom = np.asarray(ground.normals)
theta_threshold = 30
cosine_threshold = np.cos(np.deg2rad(theta_threshold))
paves = np.array([[0,0,0]])

In [130]:
from tqdm import tqdm

In [25]:
for i,p in enumerate(driving_track):
    slim = ground_np[point_neighbours[:,0] == i]
    if len(slim) > 0:
        slim_distance = np.sqrt(np.sum(np.power(slim - driving_track[i,:],2),axis = 1))
        p = slim_distance.argmin()
        slim_nom = slim - driving_track[i,:]
        slim_distance[slim_nom[:,0]/slim_nom[:,1] < 0] = slim_distance[slim_nom[:,0]/slim_nom[:,1] < 0] * -1
        slim = slim[slim_distance.argsort()]
        slim_nor = ground_nom[point_neighbours[:,0] == i]
        p = slim_nor[p,:]
        slim_nor = slim_nor[slim_distance.argsort()]
        slim_distance.sort()
        # slim_point = np.where(slim[:,:] == driving_track[i,:])[0][0] 
        a_jiao = np.fabs(np.dot(slim_nor, p))
        b = np.diff(slim[:,2])
        b =np.append(b,1)
        a = a_jiao > cosine_threshold
        # b1 = b > -0.05 
        # b2 = 0.05 > b
        # c = a & b1 & b2
        d = slim[a]
        # d = d[np.isin(d,geometrys,invert=True)]
        paves = np.concatenate((paves,d),axis=0)

C:\Users\master\AppData\Local\Temp\ipykernel_24064\3721048001.py:7: RuntimeWarning: invalid value encountered in divide
  slim_distance[slim_nom[:,0]/slim_nom[:,1] < 0] = slim_distance[slim_nom[:,0]/slim_nom[:,1] < 0] * -1


In [26]:
seed = []

In [27]:
for i in geometrys:
    a = np.argwhere((paves == i).all(axis=1)).ravel()
    if a:
        seed.append(a[0])


C:\Users\master\AppData\Local\Temp\ipykernel_24064\3883217475.py:3: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if a:


In [28]:
paves = np.delete(paves,seed,0)


In [29]:
paves = datas.np_to_o3d(paves)

In [ ]:
o3d.visualization.draw_geometries([paves])


: 

: 

In [145]:
def euclidean_cluster(cloud, tolerance=0.2, min_cluster_size=100, max_cluster_size=1000):
    """
    欧式聚类
    :param cloud:输入点云
    :param tolerance: 设置近邻搜索的搜索半径（也即两个不同聚类团点之间的最小欧氏距离）
    :param min_cluster_size:设置一个聚类需要的最少的点数目
    :param max_cluster_size:设置一个聚类需要的最大点数目
    :return:聚类个数
    """

    kdtree = o3d.geometry.KDTreeFlann(cloud)  # 对点云建立kd树索引

    num_points = len(cloud.points)
    processed = [-1] * num_points  # 定义所需变量
    clusters = []  # 初始化聚类
    # 遍历各点
    for idx in range(num_points):
        if processed[idx] == 1:  # 如果该点已经处理则跳过
            continue
        seed_queue = []  # 定义一个种子队列
        sq_idx = 0
        seed_queue.append(idx)  # 加入一个种子点
        processed[idx] = 1

        while sq_idx < len(seed_queue):

            k, nn_indices, _ = kdtree.search_radius_vector_3d(cloud.points[seed_queue[sq_idx]], tolerance)

            if k == 1:  # k=1表示该种子点没有近邻点
                sq_idx += 1
                continue
            for j in range(k):

                if nn_indices[j] == num_points or processed[nn_indices[j]] == 1:
                    continue  # 种子点的近邻点中如果已经处理就跳出此次循环继续
                seed_queue.append(nn_indices[j])
                processed[nn_indices[j]] = 1

            sq_idx += 1

        if max_cluster_size > len(seed_queue) > min_cluster_size:
            clusters.append(seed_queue)

    return clusters

In [152]:
if __name__ == '__main__':
    # --------------------------加载点云数据------------------------------
    pcd = paves
    # ---------------------------欧式聚类--------------------------------
    ec = euclidean_cluster(pcd, tolerance=0.1, min_cluster_size=1000, max_cluster_size=100000000)
    # -------------------------聚类结果分类保存---------------------------

    for i in range(len(ec)):
        ind = ec[i]
        clusters_cloud = pcd.select_by_index(ind)
        file_name = "euclidean_cluster" + str(i + 1) + ".pcd"
        o3d.io.write_point_cloud(file_name, clusters_cloud)

In [2]:
pcd = o3d.io.read_point_cloud("euclidean_cluster1.pcd")
pcd1 = o3d.io.read_point_cloud("euclidean_cluster2.pcd")
pcd2 =o3d.io.read_point_cloud("euclidean_cluster3.pcd")

In [3]:
pcd.paint_uniform_color([1, 0, 0])
pcd1.paint_uniform_color([0, 1, 0])
pcd2.paint_uniform_color([0, 0, 1])


PointCloud with 11585 points.

In [150]:
o3d.visualization.draw_geometries([pcd, pcd1, pcd2])
